The SMART metrics are the foundations of our predictive maintenance policy and, since their meaning is model dependent, the policy we derive will be different for each of them. As a first step, we read the dataset from the zipped repositories in [Backblaze](https://www.backblaze.com/b2/hard-drive-test-data.html) and identify the models which have enough entries to be suitable for the analysis.

This notebook provides a preliminary description of the dataset relative to each model. We will read the data from the zipped online repository instead of downloading the entire datasets due to their large dimensions. After the preliminary analysis we will focus on specific models, read the relative data, and save only the meaningful features (more about what we consider meaningful below).

** Note: To correctly run this notebook a good internet connection is required. By the end of the notebook the data relative to a specific model will be saved on the laptop. **


In [1]:
import numpy as np
import pandas as pd
import requests
import zipfile
import io
import os

In [2]:
# String needed to access the data in the zipfiles stored at https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data

url_15 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_2015.zip'
start_str_15 = '2015/'

url_16_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2016.zip'
start_str_16_q1 = 'data_Q1_2016/'

url_16_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2016.zip'
start_str_16_q2 = 'data_Q2_2016/'

url_16_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2016.zip'
start_str_16_q3 = 'data_Q3_2016/'

url_16_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2016.zip'
start_str_16_q4 = ''

url_17_q1 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q1_2017.zip'
start_str_17_q1 = ''

url_17_q2 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q2_2017.zip'
start_str_17_q2 = ''

url_17_q3 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q3_2017.zip'
start_str_17_q3 = ''

url_17_q4 = 'https://f001.backblazeb2.com/file/Backblaze-Hard-Drive-Data/data_Q4_2017.zip'
start_str_17_q4 = 'data_Q4_2017/'

list_url = [[[url_15,start_str_15]], 
            [[url_16_q1,start_str_16_q1], [url_16_q2,start_str_16_q2], [url_16_q3,start_str_16_q3], [url_16_q4,start_str_16_q4]], 
            [[url_17_q1,start_str_17_q1], [url_17_q2,start_str_17_q2], [url_17_q3,start_str_17_q3], [url_17_q4,start_str_17_q4]]
           ]

list_keys = [15, 16, 17]

dict_zipfile = dict(zip(list_keys,list_url))

Section A of this notebook is used to grasp the structure of the dataset with respect to the various model.

** If interested in a specific model, skip to Section B ** and run the rest of the notebook after specifying the **ModelName** variable.


# A. Number of failures per model

We will list the number of unique HDDs per model and how many of them failed in the three-year period considered. We reported only those models with at least 1000 unique HDDs and ordered them by fraction of failed HDDs. 

Note that the ratio is always below 10%, we thus have to face **class imbalance** as it is typical for anomaly detection problems. 


In [3]:
# Functions needed to read, analyze, and save the dataframes associated to the various models

def read_entries_and_failures( url, start_string = ''):
    '''This function reads the zip file located at url, open it, read the columns needed 
    to capture the per model number of entries and failures'''
    ListDF = []
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]

    for file in files : 
        data = pd.read_csv(z.open(file))
        data2 = data[['date','serial_number', 'model', 'failure']]
        ListDF.append(data2)
        
    df = pd.concat(ListDF, ignore_index = True)
    
    return df
    
    
    
def per_model_entries_and_failures (df) :   
    '''Given a dataframe with the observations and the columns 'model', 'serial_number', and 'failure, 
    return a dataframe with number of failures and unique serial numbers'''
    FailuresPerModel = df.groupby('model')['failure'].sum()
    EntriesPerModel = df.groupby('model')['serial_number'].unique()
    ModelsDetail = pd.concat([EntriesPerModel,FailuresPerModel],axis=1)
    
    return ModelsDetail
    
    
    
def save_yearly_entries_and_failures( dict_url, year ) :
    '''This function read the datasets located at dict_url associated to a specific year, it print the number of
    entries for that year, and save the dataframe containing the number of entries and failures per model'''
    list_df = []

    for url, start_string in dict_url[year] :
        df = read_entries_and_failures(url, start_string)  
        list_df = list_df + [df]
        
    if len(dict_zipfile[year]) > 1 :
        df = pd.concat(list_df, ignore_index = True)
        
    return per_model_entries_and_failures(df) 

In [4]:
# Read the dataset relative to years 2015-2017 and save the number of unique entries and failures per model

ModelsDetail_15 = save_yearly_entries_and_failures( dict_zipfile, 15 )
ModelsDetail_16 = save_yearly_entries_and_failures( dict_zipfile, 16 )
ModelsDetail_17 = save_yearly_entries_and_failures( dict_zipfile, 17 )

In [5]:
######### TO BE SKIPPED IF INTERESTED ONLY IN A SPECIFIC MODEL #########

## Combine the dataframe extracted in a unique dataframe

ListDF_raw = [ModelsDetail_15.add_suffix('_15'), ModelsDetail_16.add_suffix('_16'), ModelsDetail_17.add_suffix('_17')]
DfTot = pd.concat(ListDF_raw, axis = 1) 

# If the failure entry is equal to NaN, the value is replaced with a 0
DfTot.failure_15 = DfTot.failure_15.fillna(0) 
DfTot.failure_16 = DfTot.failure_16.fillna(0) 
DfTot.failure_17 = DfTot.failure_17.fillna(0) 

# Compute the total number of failures in the period 2015-2017
DfTot['failure_tot'] = DfTot.failure_15 + DfTot.failure_16 + DfTot.failure_17
del DfTot['failure_15']
del DfTot['failure_16']
del DfTot['failure_17']

# Merge the serial_numbers that appear in the period 2015-2017
DfTot['serial_number_tot'] = [np.hstack((DfTot.loc[mod,'serial_number_15'], DfTot.loc[mod,'serial_number_16'], DfTot.loc[mod,'serial_number_17'])) for mod in DfTot.index]

del DfTot['serial_number_15']
del DfTot['serial_number_16']
del DfTot['serial_number_17']

# Count the unique entries in the serial_number column (use the set structure to get rid of the duplicates)
DfTot['entries_tot'] = [len(set(DfTot.loc[mod,'serial_number_tot'])) for mod in DfTot.index]

del DfTot['serial_number_tot']


In [6]:
######### TO BE SKIPPED IF INTERESTED ONLY IN A SPECIFIC MODEL #########

## Compute the percentage of failed HDDs and order the models in descending order w.r.t. such percentage

DfTot['ratio_entry_failures'] = DfTot['failure_tot'].divide(DfTot['entries_tot'], axis = 'rows')
DfTot.sort_values('ratio_entry_failures', inplace=True, ascending =False)

# Display only the models with at least 1000 unique HDDs

DfTot[DfTot['entries_tot'] > 1000]
DfTot.to_csv('EntriesFailureRatio.csv')

,failure_tot,entries_tot,ratio_entry_failures
WDC WD30EFRX,122.0,1261,0.096749
ST3000DM001,106.0,1170,0.090598
ST4000DM000,2590.0,36700,0.070572
ST31500541AS,112.0,1693,0.066155
Hitachi HDS723030ALA640,44.0,1018,0.043222
Hitachi HDS722020ALA330,152.0,4683,0.032458
ST6000DX000,60.0,1938,0.030960
Hitachi HDS5C3030ALA630,96.0,4608,0.020833
Hitachi HDS5C4040ALE630,42.0,2660,0.015789
ST8000DM002,141.0,10029,0.014059


# B. Model-based data cleaning

In the following we will focus on the model **Hitachi HDS722020ALA330** and report only the result associated to such model. The code is fully automated and the same analysis can be performed for the other models as well by replacing the value assigned to **ModelName**. 

We decided to present the results for the model **Hitachi HDS722020ALA330** being it the average case. It has about 5000 entries and 3% failures, where the former ranges in between 1000 (to be suitable for analysis) and 15000 (minus one outlier at 36000) and the latter ranges from 0.2% to 9.6%.

Observe that since we consider a unique model, we can immediately drop the columns 'Model' and 'Capacity' from the dataset. For every SMART metric we have both the raw and the normalized value, for the sake of simplicity we will work only with the normalized quantities. Thus, the original dataset (with 95 columns) is immediately reduced to one with 48 columns (date, serial number, failure, and the normalized SMART metrics). We further reduce the dataset by dropping the **non-informative** SMART metrics. In particular, given a specific model some of the SMART metrics are 
absent and others do not vary significantly and thus not provide meaningful information. While the former are easy to identify, we need to provide a way to judge which metrics belong to the latter.

In [9]:
#### Select the model you want to work with. (ex:'Hitachi HDS722020ALA330', 'ST8000DM002', ...) ####

ModelName = 'Hitachi HDS722020ALA330' 
#ModelName = 'ST8000DM002' 
#ModelName = 'HGST HMS5C4040BLE640' 
#ModelName = 'WDC WD30EFRX' 
#ModelName = 'ST12000NM0007' 
#ModelName = 'Hitachi HDS5C3030ALA630' 

### B.1. Non-informative SMART metrics

Given a SMART metric, we claim that it is non-informative if it does not vary sufficiently over the dataset and is never below the typical value. 
We came up with the following method to filter-out non-informative SMART metrics. Assume that a SMART metric takes values $\mathbf{x} = x_1, \ldots, x_N$ for a specific model, we can compute 

- The variance $v(\mathbf{x}) = \text{Var}\big( x_1, \ldots, x_N \big)$;
- The range $r(\mathbf{x}) = \max\big( x_1, \ldots, x_N \big) - \min\big( x_1, \ldots, x_N \big)$;
- The minimum $m(\mathbf{x}) = \min\big( x_1, \ldots, x_N \big)$.

We claim that a metric is non-informative if

$$
 \Big( m(\mathbf{x}) \geq 100 \Big) \text{ or } \Big( \big( r(\mathbf{x}) < T_r \big) \text{ and } \big( v(\mathbf{x}) < T_v \big) \Big) 
$$

with $T_r$ and $T_v$ chosen thresholds. We selected $T_r = 15$ and $T_v = 1$.

Note that if $m(\mathbf{x}) \geq 100$ it means that the metric never takes values below the typical and thus do not provide information of eventual 
malfunctions. To capture the variability of the metric, we believe that the variance alone would not be sufficient, 
in fact in some cases the SMART metric may drop abruptly just before a failure. In such
case the large majority of the values for the metric would be close to a typical value and the variance would be too low due to the small quantity of unusual values.

The filter above is coded in the function **is_relevant** below. 

In [10]:
def is_relevant(col, thresh_diff = 15, thresh_var = 1) :
    ''' Given a column of the DataFrame, decide whether it is not informative (return False) 
    or it might be (return True)'''
    diff = col.max() - col.min()
    var = col.var()
    mincol = col.min()
    if mincol > 99 :                                # Check whether the metric is ever below the typical value 100
        return False
    if (diff > thresh_diff) | (var > thresh_var):   # Check whether the metric varies enough
        return True
    else :
        return False

The function __is_relevant__ provides a deterministic filter that could be considered as a preliminary feature selection. This does not exclude a further feature selection procedure exploiting machine learning to be used later on. 

In particular, more than saying whether a SMART metric is correlated with failures, this function simply excludes metrics that certainly aren't, i.e., the metrics that are never below the typical value or do not show significant variation in their normalized value across the dataset.

Note that the filter can be made more or less inclusive by adjusting the values of the thresholds.

In [11]:
## The following functions are needed in order to build a mask for any specific model. The mask is a filter that selects
## only the columns associated to potentially informative SMART metrics 
## (the mask excludes the metrics that are non-informative and those that are absent for a specific model)

# (1) Read the dataset associated to a model and a specific mask (only columns specified in rel_cols).
def read_model_and_rel_cols (mod_name, rel_cols, DictUrl, year):
    ''' This function reads the zip file of the selected year, open it, read the columns associated to a model and 
    the relevant columns '''
    ListDF = []
    
    for url, start_string in DictUrl[year] :
        r = requests.get(url)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        files = [name for name in z.namelist() if (name.endswith('.csv')) & (name.startswith(start_string))]
        for file in files : 
            data = pd.read_csv(z.open(file), parse_dates = [0])
            data2 = data[data['model'] == mod_name]
            data3 = data2.iloc[:,rel_cols]
            ListDF.append(data3)
    
    df = pd.concat(ListDF, ignore_index = True)
    return df


# (2) Return the SMART metrics with are present (not NaN values) and are not non-informative (is_relevant returned True).
def find_relevant_cols(df, thresh_diff = 15, thresh_var = 1) :
    ''' For the various SMART metrics return True only if the metric is considered by a specific model, i.e. the 
    relative values are not NaN and if the metric is relevant. Return the 45 boolean values list.'''
    non_na_cols = (df.isnull().sum() == 0)
        
    res_bool = []
        
    for col in df.columns :
        if non_na_cols[col] == True :
            res_bool = res_bool + [is_relevant(df[col], thresh_diff, thresh_var)]
        else :
            res_bool = res_bool + [False]
    return res_bool
    

In [12]:
## Build the mask for the model specified in ModelName.

# normalized_cols has 'False' in correspondence of a column with a raw value and 'True' of a col with a normalized value
normalized_cols = [True, False]       # The SMART metrics columns are ordered as Normalized, Raw, Normalized, Raw, ...
normalized_cols = normalized_cols*45  # ... and there are 45 SMART metrics

# The column names are ordered as [date, serial number, model, capacity, failure] + SMART metrics
cols = [True, True, False, False, True] + normalized_cols 

df_year = {}
mask = [False]*45

for year in [15,16,17] :
    # Call (1) : Read all the columns with normalized SMART metrics
    df_year[year] = read_model_and_rel_cols (ModelName, cols, dict_zipfile, year)
    # Call (2) : Find the relevant columns with the dataframe df obtained with the previous call
    mask_year = find_relevant_cols(df_year[year].iloc[:,3:], 15, 1) 
    mask = [ x | y for (x,y) in zip(mask, mask_year)]
        

### B.2. Model-based data filtering

We now use the filter created to select only the relevant columns for the model in ModelName. The use of the mask allows a drastic reduction in the dimensions of the dataset making it way more manageable while retaining most the information that could lead to the early prediction of an imminent failure. 

**For the model Hitachi HDS722020ALA330 we drop most of the metrics and keep only the columns associated to the normalized SMART metrics number 1, 5, 7, 8, 192, 193, and 196**, i.e., 7 out of the initial 45 metrics. 

We now save the model-based filtered dataframe in a dataset that will be used in the following notebooks to analyze the relation between SMART metrics and time-to-failure.

In [13]:
# Path to the folder in which the datset will be stored
folder_name_model = ModelName.replace(' ', '_')
folder_name = 'Data/' + folder_name_model

# Check whether the folder exists and create it if it does not. 
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# We drop the columns relative to model, capacity, and non-informative SMART metrics
cols_chosen = [True, True, True] + mask      # [date, serial number, failure] + chosen SMART metrics     
    
# Save one separate dataframe per year
for year in [15,16,17] :
    df_year_chosen = df_year[year].iloc[:, cols_chosen]
    df_year_chosen.to_csv(folder_name + '/Model_' + str(year) + '.csv')
    